# Introducing Langfuse SDK - Solution

This notebook demonstrates how to integrate the Langfuse SDK into a Python application to trace and observe language model calls using the low-level SDK. We will use the GPT-2 text generation model from the `transformers` library as an example.

This approach provides fine-grained control over tracing individual LLM calls.

## Sign Up for Langfuse

Before you begin, you need to sign up for a free Langfuse account to get your API keys. Follow these steps:

1.  Go to the Langfuse website: [https://langfuse.com/](https://langfuse.com/)
2.  Click on "Sign Up".
3.  Follow the instructions to create your account.
4.  Once logged in, navigate to your project settings to find your **Public Key** and **Secret Key**. You will also need your **Host** URL (usually `https://cloud.langfuse.com`).
5.  Keep these keys handy. It is highly recommended to set these as environment variables (`LANGFUSE_PUBLIC_KEY`, `LANGFUSE_SECRET_KEY`, `LANGFUSE_HOST`) before running this notebook for security. Research how to set environment variables in your operating system or within a Jupyter environment.

## 1. Installation

We need to install the `langfuse` SDK, `transformers`, and `torch`.

In [ ]:
# Install the required libraries
!pip install langfuse transformers torch

## 2. Langfuse and Model Setup

Import the necessary libraries and initialize Langfuse. Langfuse will automatically pick up your API keys and host from environment variables if they are set. We will also load the GPT-2 model using the `transformers` pipeline.

In [ ]:
import os
from langfuse import Langfuse
from transformers import pipeline, set_seed

# Initialize Langfuse. It reads keys from environment variables by default.
# Ensure LANGFUSE_PUBLIC_KEY, LANGFUSE_SECRET_KEY, and LANGFUSE_HOST are set.

%env LANGFUSE_SECRET_KEY=sk-lf-30d71f39-9615-4e6b-a0f7-ec1b8d14acf7
%env LANGFUSE_PUBLIC_KEY=pk-lf-e3e8cfc3-e32e-4b9b-a02f-53ada416db02
%env LANGFUSE_HOST=https://cloud.langfuse.com

try:
    langfuse = Langfuse()
    print("Langfuse initialized successfully.")
except Exception as e:
    print(f"Error initializing Langfuse: {e}")
    print("Please ensure LANGFUSE_PUBLIC_KEY, LANGFUSE_SECRET_KEY, and LANGFUSE_HOST environment variables are set correctly.")
    langfuse = None # Set to None if initialization fails

# Initialize GPT-2 model
# Using a pipeline simplifies text generation
generator = pipeline('text-generation', model='gpt2')
set_seed(42) # Set seed for reproducibility

print("GPT-2 model loaded.")

## 3. Tracing with Langfuse (Low-Level SDK)

We will now trace a single text generation call using the Langfuse low-level SDK. This involves creating a `generation` object before the LLM call and ending it with the output afterwards. This method gives explicit control over what is traced.

In [ ]:
# Define the input prompt
prompt = "The quick brown fox jumps over the lazy dog because"

try:
    # Create a trace
    trace = langfuse.trace(
        name="GPT-2 Trace",
        input=prompt,
        metadata={"task": "text-completion"}
    )

    # Create a generation associated with the trace
    generation = trace.generation(
        name="GPT-2 Generation",
        model="gpt2",
        model_parameters={
            "max_length": 100,
            "num_return_sequences": 1
        },
        input=[{"role": "user", "content": prompt}],
        metadata={"task": "text-completion"}
    )

    # Generate output using GPT-2
    output = generator(
        prompt,
        max_length=100,
        num_return_sequences=1,
        pad_token_id=generator.tokenizer.eos_token_id
    )[0]['generated_text']

    # End the generation with the output
    generation.end(output=output)

    # Update the trace with the output
    trace.update(output=output)

    print(f"Generated Text:\n{output}")
    print("Check the Langfuse UI for the trace.")

except Exception as e:
    # Handle exceptions and end the generation with error status
    if 'generation' in locals() and generation:
        generation.end(level="ERROR", status="ERROR", extra={"error_message": str(e)})
    print(f"An error occurred during text generation or tracing: {e}")

# Flush Langfuse events
print("Flushing Langfuse events...")
langfuse.flush()
print("Events flushed.")

## 4. View the Trace in Langfuse

After running the previous cell (and if Langfuse was initialized successfully), a trace for the GPT-2 generation will be sent to your Langfuse project. Go to your Langfuse project UI ([https://cloud.langfuse.com/](https://cloud.langfuse.com/) or your self-hosted instance) to view the trace details. You should see the input prompt, the generated output, model parameters, metadata, and timing information.